In [3]:
import geemap

In [4]:
import geopandas as gpd
import pandas as pd 
import numpy as np 

In [5]:
geemap.set_proxy(port = 1081)

In [ ]:
# geemap.update_package()

In [6]:
import ee
ee.Authenticate()


Successfully saved authorization token.


In [7]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [8]:
points = ee.FeatureCollection('users/xtnncherish/nj_points')
Map.addLayer(points, {}, "points", False)

In [24]:
lulc = ee.ImageCollection("MODIS/006/MCD12Q1").filterDate('2018-01-01','2019-01-01').select('LC_Type1').mosaic()
dem = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
ndvi = ee.ImageCollection("MODIS/006/MOD13Q1").filterDate('2019-06-01','2019-11-01').select('NDVI').median()
lst = ee.ImageCollection("JAXA/GCOM-C/L3/LAND/LST/V1") .filterDate('2019-06-01','2019-11-01').select('LST_AVE').median()

In [25]:
Map.addLayer(lst,{},'LST')

In [26]:
slope = ee.Terrain.slope(dem)

In [27]:
data_list = [lulc, slope, ndvi, lst]
data_name = ['lulc', 'slope', 'ndvi', 'lst']
data_scale = [500, 30, 100, 500]
for i in range(4):
    work_dir = os.path.expanduser('~/Downloads')
    out_shp = os.path.join(work_dir,'points_{}.shp'.format(data_name[i]))
    geemap.extract_values_to_points(in_fc = points, image = data_list[i], scale =data_scale[i], out_fc = out_shp)

# work_dir = os.path.expanduser('~/Downloads')
# out_shp = os.path.join(work_dir, 'points_lulc.shp')
# geemap.extract_values_to_points(in_fc=points, image=lulc, out_fc = out_shp)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_lulc.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_slope.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_ndvi.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_lst.shp


In [11]:
work_dir = os.path.expanduser('~/Downloads')
out_shp = os.path.join(work_dir,'points_ndvi.shp')
geemap.extract_values_to_points(in_fc = points, image = ndvi, scale = 100, out_fc = out_shp)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_ndvi.shp


In [28]:
points_lulc = gpd.read_file(r'C:\Users\Admin\Downloads\points_lulc.shp')
points_slope = gpd.read_file(r'C:\Users\Admin\Downloads\points_slope.shp')
points_ndvi = gpd.read_file(r'C:\Users\Admin\Downloads\points_ndvi.shp')
points_lst = gpd.read_file(r'C:\Users\Admin\Downloads\points_lst.shp')

In [33]:
print(points_lst.head())
# print(points_slope.head())
# print(points_ndvi.head())

       OBJECTID      COR_Y  LST_AVE      COR_X                   geometry
6850          1  41.340586  14732.0  79.960606  POINT (79.96129 41.34272)
13952         2  41.340799  14732.0  79.984496  POINT (79.98375 41.34272)
14684         3  41.377420  14629.0  80.055649  POINT (80.05561 41.37865)
392           4  41.377614  14629.0  80.079553  POINT (80.07807 41.37865)
12501         5  41.377803  14646.0  80.103458  POINT (80.10502 41.37865)


In [30]:
points_lulc.sort_values('OBJECTID', inplace = True)
points_slope.sort_values('OBJECTID', inplace = True)
points_ndvi.sort_values('OBJECTID', inplace = True)

In [31]:
points_lst.sort_values('OBJECTID', inplace = True)

In [37]:
nj_points = gpd.read_file(r'F:\python_code\gdal_gee\GEE_LEARN\nj_points.shp')
nj_points.head()

,OBJECTID,COR_Y,COR_X,geometry
0,1,41.340586,79.960606,POINT (79.96061 41.34059)
1,2,41.340799,79.984496,POINT (79.98450 41.34080)
2,3,41.377420,80.055649,POINT (80.05565 41.37742)
3,4,41.377614,80.079553,POINT (80.07955 41.37761)
4,5,41.377803,80.103458,POINT (80.10346 41.37780)


In [38]:
nj_points['LULC'] = points_lulc['LC_Type1'].values
nj_points['SLOPE'] = points_slope['slope'].values
nj_points['NDVI'] = points_ndvi['NDVI'].values
nj_points['LST'] = points_lst['LST_AVE'].values

In [39]:
nj_points.head()

,OBJECTID,COR_Y,COR_X,geometry,LULC,SLOPE,NDVI,LST
0,1,41.340586,79.960606,POINT (79.96061 41.34059),10,1.544394,3178.0,14732.0
1,2,41.340799,79.984496,POINT (79.98450 41.34080),15,3.086556,699.0,14732.0
2,3,41.377420,80.055649,POINT (80.05565 41.37742),10,2.780288,2347.0,14629.0
3,4,41.377614,80.079553,POINT (80.07955 41.37761),10,0.927410,5296.0,14629.0
4,5,41.377803,80.103458,POINT (80.10346 41.37780),10,1.544959,2647.0,14646.0


In [21]:
nj_points.crs

<Geographic 2D CRS: EPSG:4610>
Name: Xian 1980
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: China - onshore
- bounds: (73.62, 18.11, 134.77, 53.56)
Datum: Xian 1980
- Ellipsoid: IAG 1975
- Prime Meridian: Greenwich

In [22]:
import os
os.getcwd()

'f:\\python_code\\gdal_gee\\GEE_LEARN'

In [40]:

nj_points.to_file(driver = 'ESRI Shapefile', filename = 'nj_points_value0.shp')

# gdf.to_file(driver = 'ESRI Shapefile', filename = input_path)